### Import packages

Import of the packages that will be needed for the project.  This includes packages for data manipulation, sklearn modules and custom functions.

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, MinMaxScaler, FunctionTransformer

from Project_Functions import offensive_contribution, trailing_stats_mean, tier_maker, get_tiers, get_touchdowns, stats_for_trailing

In [2]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action = 'ignore',
                     category = SettingWithCopyWarning)

### Import Data

Let's import the dataframe that we will be using for modelling

In [3]:
data = pd.read_csv('Data/weekly_data.csv')
data = data[data['FantasyPointsPPR'] != 0]
data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade40to49,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0


In [4]:
data.isna().sum()

Unnamed: 0      0
Unnamed: 0.1    0
GameKey         0
PlayerID        0
Season          0
               ..
HomePoints      0
HomeYards       0
AwayTeam        0
AwayPoints      0
AwayYards       0
Length: 65, dtype: int64

Before the train test split, we have to calculate the trailing average fantasy points for each observation, as we cannot incorporate this step into the pipeline without causing data leakage.

In [5]:
def trailing_stats(df):
    
    """
    Function to create a new column with a trailing aggregate mean
    as a new feature for prediction.
    
    Inputs:
        - df: The dataframe on which the function will be applied
        - Column: The column on which to apply the function
        - Window: The number of past values to consider when apply the function

        
    Output:
        - An aggregate value
        
    """
    #Access the column names in stats_for_trailing
    global stats_for_trailing
    
    # Get all unique players in the DataFrame
    players = df['Name'].unique().tolist()
    
    # Define a DataFrame to hold our values
    df_out = pd.DataFrame()
    # Loop through the unique players
    for player in players:
        # Create a temporary dataframe for each player
        temp_df = df[df['Name'] == player]
        # Calculate the n game trailing average for all players.  Set closed parameter to 'left'
        # so that the current value for fantasy points is not included in the calculation.
        # Backfill the two resulting NaN values
        for column in stats_for_trailing:
            temp_df[f'TA{column}'] = temp_df.loc[:,column].rolling(window = 5, 
                                                              closed = 'left').mean().fillna(method = 'bfill')
        # Append the temporary dataframe to the output
        df_out = df_out.append(temp_df)
    # Return a dataframe with the values sorted by the original index
    return df_out.sort_index()
    

In [6]:
stats_for_trailing = ['FantasyPointsPPR']

In [7]:
# Prepare the trailing average fantasy points column

data = trailing_stats(data)
data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards,TAFantasyPointsPPR
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,24.000
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,28.600
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,25.204
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0,18.240
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0,22.420


### Train Test Split

In [8]:
# Separate data from the target
y = data['FantasyPointsPPR']
data.drop(columns = ['FantasyPointsPPR'],
            inplace = True)
X = data

In [9]:
# Execute the train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.25,
                                                   random_state = 13)

### Feature Engineering

The main features that we will be engineering to predict a player's fantasy output will be the 5-game trailing average of various statistics as well as the binning of players into their respective tiers based on recent performance.

In [10]:
# Define the columns for which we want a 5 game trailing average.

stats_for_trailing = ['TotalTouchdowns','RushingYards','PassingInterceptions','PassingTouchdowns','PassingRating','PassingYards',
                      'PassingCompletionPercentage','RushingYards', 'RushingTouchdowns', 'RushingLong',
                      'Receptions','ReceivingYards','ReceivingTargets', 'ReceivingTouchdowns', 'ExtraPointsMade', 'FieldGoalsMade',
                      'Fumbles','FumblesLost']

trailing_stats = []
for col in stats_for_trailing:
    trailing_stats.append('TA' + col)
trailing_stats

['TATotalTouchdowns',
 'TARushingYards',
 'TAPassingInterceptions',
 'TAPassingTouchdowns',
 'TAPassingRating',
 'TAPassingYards',
 'TAPassingCompletionPercentage',
 'TARushingYards',
 'TARushingTouchdowns',
 'TARushingLong',
 'TAReceptions',
 'TAReceivingYards',
 'TAReceivingTargets',
 'TAReceivingTouchdowns',
 'TAExtraPointsMade',
 'TAFieldGoalsMade',
 'TAFumbles',
 'TAFumblesLost']

In [11]:
# Instantiate the function transformers for the feature engineering pipeline

touchdown_transformer = FunctionTransformer(get_touchdowns) # Get total touchdowns per player
trailing_transformer = FunctionTransformer(trailing_stats_mean) # Get the 5 game trailing averages of appropriate statistics
tier_transformer = FunctionTransformer(get_tiers) # Bin players into the appropriate tiers based on recent performance


# Instantiate the pipeline for the necessary transformations

engineering = Pipeline([('touchdown', touchdown_transformer),
                       ('trailing', trailing_transformer),
                       ('tier', tier_transformer)])

In [12]:
test = engineering.fit_transform(X_train)

In [13]:
test

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,TARushingLong,TAReceptions,TAReceivingYards,TAReceivingTargets,TAReceivingTouchdowns,TAExtraPointsMade,TAFieldGoalsMade,TAFumbles,TAFumblesLost,PlayerTier
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,11.6,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.2,QB4
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,22.8,4.8,43.0,6.4,0.2,0.0,0.0,0.0,0.0,RB1
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,QB1
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,17.0,3.4,20.0,4.6,0.0,0.0,0.0,0.4,0.2,RB1
5,6,6,201810129,8244,2018,2018-09-09T16:05:00,1,LAC,KC,HOME,...,2.8,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,QB4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31448,101475,1179,202111034,22591,2021,2021-11-14T13:00:00,10,TEN,NO,HOME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WR4
31533,101821,1525,202111025,2405,2021,2021-11-14T20:20:00,10,KC,LV,AWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QB4
31535,101823,1527,202111012,22569,2021,2021-11-14T16:25:00,10,SEA,GB,AWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WR4
31536,101824,1528,202111019,16003,2021,2021-11-11T20:20:00,10,BAL,MIA,AWAY,...,0.0,3.6,58.0,6.4,0.0,0.0,0.0,0.0,0.0,WR3


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13882 entries, 0 to 31537
Data columns (total 84 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     13882 non-null  int64  
 1   Unnamed: 0.1                   13882 non-null  int64  
 2   GameKey                        13882 non-null  int64  
 3   PlayerID                       13882 non-null  int64  
 4   Season                         13882 non-null  int64  
 5   GameDate                       13882 non-null  object 
 6   Week                           13882 non-null  int64  
 7   Team                           13882 non-null  object 
 8   Opponent                       13882 non-null  object 
 9   HomeOrAway                     13882 non-null  object 
 10  Number                         13882 non-null  int64  
 11  Name                           13882 non-null  object 
 12  Position                       13882 non-null 

In [15]:

test[test['TATotalTouchdowns'].isna()].groupby('Name')['Name'].value_counts().sort_values()

Name                Name              
Joe Reed            Joe Reed              1
Garrett Celek       Garrett Celek         1
Shelton Gibson      Shelton Gibson        1
Greg Dortch         Greg Dortch           1
Gregory Howell Jr.  Gregory Howell Jr.    1
                                         ..
Demetric Felton     Demetric Felton       5
Marcell Ateman      Marcell Ateman        5
Joshua Palmer       Joshua Palmer         5
Daniel Brown        Daniel Brown          5
Evan McPherson      Evan McPherson        5
Name: Name, Length: 359, dtype: int64

<br>

### Preprocessing

As shown above, the bulk of the null values fall into one of two categories.  They are either:
* In the InjuryStatus column
    * Here we can impute a value of healthy, as the only values in the injury column are 
* In the TA (trailing average) columns we created
    * No player with a null value played more than 5 games, therefore we cannot calculate the trailing average for them.  We will impute a default value of 0 for these columns, as they represent players who likely did not have much impact.  If they had an impact, they likely would have played in more games.  I will explore imputing the median value as well through a grid search.

In [16]:
# Define the groups of columns for preprocessing steps.

categorical_columns = ['Week',
                       'Team',
                      'Opponent',
                      'PlayerTier',
                      'InjuryStatus']
numerical_columns = trailing_stats


In [17]:
# Define the preprocessing steps for categorical features
categorical_transform = Pipeline([('impute_cat',SimpleImputer(strategy = 'constant',
                                                              fill_value = 'Healthy')),
                                 ('one_hot_encoder', OneHotEncoder())])

# Define the preprocessing steps for numerical features
numerical_transform = Pipeline([('impute_num', SimpleImputer(strategy = 'constant',
                                                             fill_value = 0)),
                               ('scaler', StandardScaler())])

# Instantiate the column transformer object for the preprocessing steps
preprocessing = ColumnTransformer([('num', numerical_transform, numerical_columns),
                                  ('cat', categorical_transform, categorical_columns)])

In [18]:
# Instantiate a pipeline with a linear regression model as a baseline

pipeline = Pipeline([('engineering', engineering),
                    ('prep', preprocessing),
                    ('model', LinearRegression())])

In [19]:
# Set param grid values, parameters for grid search 

param_grid = {'prep__num__scaler': [StandardScaler(), 
                                             RobustScaler(),
                                            MinMaxScaler()]}

grid_search = GridSearchCV(pipeline, 
                          param_grid = param_grid,
                          cv = 5)

In [ ]:
# Fit the grid search to X_train and y_train

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

In [ ]:
preds = grid_search.predict(X_test)

In [ ]:
mean_squared_error(y_test, preds)

In [ ]:
mean_absolute_error(y_test, preds)

Let's dig into the predictions a little bit

In [ ]:
df_preds = pd.DataFrame(y_test)

In [ ]:
df_preds['Predicted'] = preds

In [ ]:
df_preds[:50]

### Save model


In [ ]:
import pickle

pickle.dump(grid_search, open('Pickles/baseline.pickle', 'wb'))